In [ ]:
'''
version 2
去除 face mask
2017年10月17日15:53:14
    在预训练模型的基础下，对现实情形进行标定校正
'''

In [ ]:
import tensorflow as tf
import os
import numpy as np
from mit_eye_v2 import mit_itraker
import time
sess=tf.InteractiveSession()

In [ ]:
#回归问题与分类问题的 y的形状不一样
with tf.name_scope('PlaceHolder'):
    LeftEye=tf.placeholder(tf.float32,[None,64,64,3],name='LeftEye')
    RightEye=tf.placeholder(tf.float32,[None,64,64,3],'RightEye')
    FaceOri=tf.placeholder(tf.float32,[None,64,64,3],'FaceOri')
    #********************** 回归为 2
    Y=tf.placeholder(tf.float32,[None,16],name='Y')
    #********************** 分类为 C
    LR=tf.placeholder(tf.float32,name='Learning_rate')
    REG_RATE=tf.placeholder(tf.float32,name='Reg_rate')
    DROPOUT=tf.placeholder(tf.float32,name='DropOut_rate')
    BN_FLAG=tf.placeholder(tf.bool,name='BatchNormalizeFlag')

In [ ]:
with tf.name_scope('MIT_Model'):
    model=mit_itraker(r'D:\Data warehouse\Face_Picture\eye_tracker_train_and_val',left_eye=LeftEye,right_eye=RightEye,face_ori=FaceOri,\
                      dropout=DROPOUT,bn_train=BN_FLAG,reg_rate=REG_RATE,load_trainimg=False)
    y_pre=model.coordinate

In [ ]:
#原 MIT数据训练的模型
saver=tf.train.Saver()
saver.restore(sess,'./model_save/model_mit/model_err112.ckpt-50030')

In [ ]:
model.calculate_class()
y_class=model.class_op
#重置FC层
model.reset_fclayer(sess,init_newclasslayer=True)

In [ ]:
#回归问题的 loss
# with tf.name_scope('LOSS'):
#     
#     Loss=(tf.losses.mean_squared_error(Y,y_pre))+tf.add_n(tf.get_collection('loss_w'))
#     
#     tf.summary.scalar('loss',Loss)
#     ERR=tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.squared_difference(y_pre, Y), axis=1)))
#     tf.summary.scalar('err',ERR)
# with tf.name_scope('Trainer'):
#     Trainer=tf.train.AdamOptimizer(LR).minimize(Loss)

# with tf.name_scope('Logger'):
#     init=tf.global_variables_initializer()

#     writer_tr=tf.summary.FileWriter('./mylog/train_nofg_cal_10181130',sess.graph)
#     writer_te=tf.summary.FileWriter('./mylog/val_nofg_cal_10181130')
#     merge=tf.summary.merge_all()

In [ ]:
#分类问题
with tf.name_scope('LOSS'):
    Loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=y_class))\
    +tf.add_n(tf.get_collection('loss_w'))
    tf.summary.scalar('loss',Loss)
    ACC_C=tf.equal(tf.arg_max(Y,1),tf.arg_max(y_class,1))
    ACCURACY=tf.reduce_mean(tf.cast(ACC_C,tf.float32))
    tf.summary.scalar('err',ACCURACY)
with tf.name_scope('Trainer'):
    Trainer=tf.train.MomentumOptimizer(learning_rate=LR,use_nesterov=True,momentum=0.8,).minimize(Loss)

with tf.name_scope('Logger'):
    init=tf.global_variables_initializer()

    writer_tr=tf.summary.FileWriter('./mylog/train_nofg_cal_clssify',sess.graph)
    writer_te=tf.summary.FileWriter('./mylog/val_nofg_cal_clssify')
    merge=tf.summary.merge_all()

In [ ]:
# data=model.data
#model.data=None
bn_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)
'''
['val_face', 'train_eye_left', 'train_y', 'train_eye_right', 'val_y', 'train_face_mask', 'train_face',
'val_eye_right', 'val_face_mask', 'val_eye_left']
'''

In [ ]:
#重新训练
init=tf.global_variables_initializer()
sess.run(init)
saver=tf.train.Saver()

In [ ]:
#加载保存的模型，继续训练
saver=tf.train.Saver()
#saver=tf.train.Saver()

#校准后的模型
saver.restore(sess,'./model_save/model_mit_cal_1017/model_10181422.ckpt-14194')

#原 MIT数据训练的模型
#saver.restore(sess,'./model_save/model_mit/model_err112.ckpt-50030')

In [ ]:
# val_left_eye,val_right_eye,val_face_matrix,val_y_cordinate=left_eye[te_index],right_eye[te_index],face_matrix[te_index],y_cordinate[te_index]
# tr_left_eye,tr_right_eye,tr_face_matrix,tr_y_cordinate=left_eye[tr_index],right_eye[tr_index],face_matrix[tr_index],y_cordinate[tr_index]

In [ ]:
cla_data=np.load('caldata_1018.pkl')

In [ ]:
cla_data.keys()

In [ ]:

index=np.arange(cla_data['face'].shape[0])
np.random.shuffle(index)
tr_index=index[:int(index.shape[0]*0.7)]
te_index=index[int(index.shape[0]*0.7):]
val_left_eye,val_right_eye,val_face_matrix,val_y_cordinate=\
cla_data['left_eye'][te_index],cla_data['right_eye'][te_index],cla_data['face'][te_index],cla_data['y'][te_index]
tr_left_eye,tr_right_eye,tr_face_matrix,tr_y_cordinate=\
cla_data['left_eye'][tr_index],cla_data['right_eye'][tr_index],cla_data['face'][tr_index],cla_data['y'][tr_index]


In [ ]:
tr_left_eye.shape,val_left_eye.shape

In [ ]:
#130 个loop用时1分钟 无BN,dropout
#5000 个loop用时1h 有BN,dropout
batch_size=128
dp_=0.5
lr_=1e-3
rg_rate_=1e-4
s_time=time.time()
best_val_err=100
err_delta=0.05
for i_ in range(5000*15):
    mask=np.random.choice(range(tr_left_eye.shape[0]),batch_size,replace=False)
    face_=tr_face_matrix[mask]
    left_eye_,right_eye_=tr_left_eye[mask],tr_right_eye[mask]
    y_=tr_y_cordinate[mask]
    
    feed_dict={LeftEye:left_eye_,  RightEye:right_eye_,  FaceOri:face_,  \
              Y:y_,LR:lr_,  DROPOUT:dp_,  BN_FLAG:True,  REG_RATE:rg_rate_}
    sess.run([Trainer,bn_ops],feed_dict=feed_dict)
    if i_%10==0:
        feed_dict={LeftEye:left_eye_,RightEye:right_eye_,FaceOri:face_,\
              Y:y_,LR:lr_,DROPOUT:1.,BN_FLAG:False,REG_RATE:rg_rate_}
        loss_,acc_,m_,_=sess.run([Loss,ACCURACY,merge,bn_ops],feed_dict=feed_dict)
        writer_tr.add_summary(m_,i_)
        print('epoch:{},loss:{},accuracy:{}'.format(i_,loss_,acc_))
    if i_%50==0:
        #mask=np.random.choice(range(5000),batch_size,replace=False)
        face_=val_face_matrix
        left_eye_,right_eye_=val_left_eye,val_right_eye
        y_=val_y_cordinate
        feed_dict={LeftEye:left_eye_,RightEye:right_eye_,FaceOri:face_,\
                  Y:y_,LR:lr_,DROPOUT:1.,BN_FLAG:False,REG_RATE:rg_rate_}
        loss_,acc_,m_,_=sess.run([Loss,ACCURACY,merge,bn_ops],feed_dict=feed_dict)
        writer_te.add_summary(m_,i_)
        print('--epoch:{},loss:{},error:{}'.format(i_,loss_,acc_))
        #保存模型
#         if err_<0:
#             if err_+err_delta<best_val_err:
#                 best_val_err=err_
#                 err_val=str(int(err_*100))
#                 saver.save(sess,'./model_save/model_mit/model_err'+err_val+'.ckpt',global_step=i_)
#                 print('@@@epoch:{},best err:{},model save'.format(i_,best_val_err))
#     if i_%(5000)==0:
#         lr_=max(1e-5,lr_*0.9)
#         print("**epoch:{},lr:{}".format(i_,lr_))
e_time=time.time()
run_time=e_time-s_time
print('Trainning Time:{:.0f} h,{:.0f} min,{:.2f} sec'.format(run_time//3600,(run_time%3600)//60,run_time%60))


In [ ]:
saver.save(sess,'./model_save/model_mit_cal_1017/model_10181422'+'.ckpt',global_step=i_)

In [ ]:
/*********************************************************************************/
#测试运行效果
/*********************************************************************************/

In [ ]:
def drawline(img_,line_w=1,line_color=(0,0,0),wandh_num=4):
        '''
        在图片上格子
        :param line_w: 线宽
        :param line_color: 线颜色
        :param wandh_num:  长宽线的数量
        :return:  无
        '''
        h,w=img_.shape[0],img_.shape[1]
        w_num,h_num=wandh_num,wandh_num
        h_,w_=h//h_num,w//w_num

        # 竖线 (w,h)
        for i in range(1,w_num):
            #print(i)
            cv2.line(img_,(w_*i,0),(w_*i,h),line_color,line_w)
        # 横线
        for i in range(1,h_num):
            cv2.line(img_,(0,h_*i),(w,h_*i),line_color,line_w)
        return img_
def drawblock(img,block_id=(3,3),blockcolor=(210,240,50),blockwideth=5,wandh_num=4):
        '''
        选定九宫格，在这个格子上填充矩形表示选定这个格子
        :param img_: 图片
        :param block: 九宫格序号
        :param blockcolor: 矩形框颜色
        :param blockwideth: 框的宽度
        :return:
        '''
        h,w=img.shape[0],img.shape[1]
        w_line,h_line=wandh_num,wandh_num
        h_,w_=h//h_line,w//w_line
        sx,sy=block_id[0]*w_,block_id[1]*h_

        #将整个矩形填充为其他颜色
        img[sy:sy+h_,sx:sx+w_,:]=blockcolor
        #在矩形中心画一个小圆辅助
        roi_=img[sy:sy+h_,sx:sx+w_]
        cv2.circle(roi_,(roi_.shape[1]//2,roi_.shape[0]//2), 10, (130,230,220), -1)

        #只是在矩形边缘画框
        #cv2.rectangle(img_,(sx,sy),(sx+w_,sy+h_),blockcolor,blockwideth)
        return img

In [ ]:
import cv2
import time

In [ ]:
tar_img=cv2.imread('test.jpg')
tar_img=drawline(tar_img,wandh_num=4)
tar_img=drawblock(tar_img,wandh_num=4,block_id=[2,2])
cap=cv2.VideoCapture(0)
_,frame=cap.read()
cap.release()
v_test=tar_img.copy()
tar_img[:480,1280:]=frame

In [ ]:
cv2.namedWindow('Capture001',cv2.WINDOW_NORMAL)
cv2.setWindowProperty('Capture001', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
cv2.imshow('Capture001',tar_img)
cv2.waitKey()
cv2.destroyWindow('Capture001')

In [ ]:
fps=10
#fsize_people=(640,480)
fsize_desk=(1920,1080)
# save video
#video_p=cv2.VideoWriter('demo_people.flv',cv2.VideoWriter_fourcc('F','L','V','1'),fps,fsize_people)
video_d=cv2.VideoWriter('demo_desk.flv',cv2.VideoWriter_fourcc('F','L','V','1'),fps,fsize_desk)

In [ ]:
#%debug
cap=cv2.VideoCapture(0)
s_time=time.time()
tar_img=cv2.imread('test.jpg')
#设置window 为全屏
cv2.namedWindow('Capture001',cv2.WINDOW_NORMAL)
cv2.setWindowProperty('Capture001', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
while True:
    ret,fram=cap.read()
    if ret:
        #显示 内窥镜图像
        tar_img=cv2.imread('test.jpg')
        tar_img=drawline(tar_img,wandh_num=4)
        cv2.imshow('Capture001',tar_img)
       
        #预测注视方向
        face_op=drew_face_eye(fram)
        if face_op == None:
            continue
        img_,f,e=face_op
        #需要 输入均值为 0 ，方差为 1
        left_eye_=(cv2.resize(e[0],(64,64))-mean_left_eye).reshape((1,64,64,3))
        right_eye_=(cv2.resize(e[1],(64,64))-mean_right_eye).reshape((1,64,64,3))
        face_=(cv2.resize(f,(64,64))-mean_face_matrix).reshape((1,64,64,3))
        
        feed_dict={LeftEye:left_eye_/255.,RightEye:right_eye_/255.,FaceOri:face_/255.,\
                 DROPOUT:1.,BN_FLAG:False}
        y_cori=sess.run(y_pre,feed_dict=feed_dict)
        y_cori=[int(y_cori[0][0]),int(y_cori[0][1])]
       
        #print(y_cori)
        #显示预测方向
        tar_img=drawblock(tar_img,wandh_num=4,block_id=y_cori)
        v_img=tar_img.copy()
        v_img[:480,1280:]=fram
        cv2.imshow('Capture001',tar_img)
       
        #cv2.imshow('asd',fram)
        
        #video_p.write(fram)
        video_d.write(v_img[:,::-1,:])
        if cv2.waitKey(30)&0xff==27:
            break
    if (time.time()-s_time)>60*2:
        print('time out')
        break
#video_p.release()
video_d.release()
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
/****************************************************************************/
获取标定图片
*****************************************************************************/

In [ ]:
import cv2
import interface
import os
import numpy as np
import pickle

In [ ]:

def drew_face_eye(img):
        '''
        检测用户当前环境是否能够比较好的识别面部特征
        辨识出脸部以及眼部，标记出来
        :param img:
        :return:
        '''
        face_cascade = cv2.CascadeClassifier(r'D:\Proj_DL\Code\Proj_EyeTraker\haarcascade_frontalface_default.xml')
        eye_cascade = cv2.CascadeClassifier(r'D:\Proj_DL\Code\Proj_EyeTraker\haarcascade_eye.xml')
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3,5)
        #确保画面中只有一个人脸识别出来
        if(len(faces)!=1):
            return None
        for (x,y,w,h) in faces:
            cv2.rectangle(img,(x-10,y-10),(x+w+10,y+h+10),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
            #保存脸部图像
            face_mat=roi_color.copy()
            #检测视频中脸部的眼睛，并用vector保存眼睛的坐标、大小（用矩形表示）
            eyes = eye_cascade.detectMultiScale(roi_gray,scaleFactor=1.3, minNeighbors=7, minSize=(50, 50),
                                                flags=cv2.CASCADE_SCALE_IMAGE)
            if len(eyes)!= 2:
                return None
            eye_mat=[]
            for e in eyes:
                xe,ye,we,he=e
                eye_mat.append(face_mat[ye:ye+he,xe:xe+we])
                cv2.rectangle(roi_color,(xe-3,ye-3),(xe+we+3,ye+he+3),(0,0,255),2)
        return  (img,face_mat,eye_mat)

In [ ]:
#开始标定
cal_obj=interface.Cal_Interface(line_num=4,cal_time=16*2)
cal_obj.starcalibrate()

In [ ]:
file_root=r'D:\Proj_DL\Code\Proj_EyeTraker\Proj_iTraker\MIT_iTraker\calimg_file_num'
img_names=os.listdir(file_root)

In [ ]:
#array([ 1.70154929, -1.93875003], dtype=float32) 标签格式
y_cordinate=[]
left_eye=[]
right_eye=[]
face_matrix=[]
for f in img_names:
    #x_,y_=f[:-4].split('_')[-2:]
    #x_,y_=int(x_[-1]),int(y_[-1])
    lable_id=int(f[:-4].split('_')[-1])
    img=cv2.imread(os.path.join(file_root,f))
    face_op=drew_face_eye(img)
    if face_op == None:
        #print('fail')
        continue
    img_,f,e=face_op
    lable_=[0]*16
    lable_[lable_id]=1
    y_cordinate.append(lable_)
    #print(lable_)
    left_eye.append(cv2.resize(e[0],(64,64)))
    right_eye.append(cv2.resize(e[1],(64,64)))
    face_matrix.append(cv2.resize(f,(64,64)))
    #cv2.imshow('asd',img_)
    if cv2.waitKey(1)&0xff == 27:
        break
cv2.destroyAllWindows()
y_cordinate=np.array(y_cordinate).astype('float32')

left_eye=np.array(left_eye)
left_eye=left_eye.astype('float32')-np.mean(left_eye,0)
left_eye/=255.0

right_eye=np.array(right_eye)
right_eye=right_eye.astype('float32')-np.mean(right_eye,0)
right_eye/=255.0

face_matrix=np.array(face_matrix)
face_matrix=face_matrix.astype('float32')-np.mean(face_matrix,0)
face_matrix/=255.0



In [ ]:
pickle_dataname=
#保存 方差为 1 ，均值为 0 的数据
cal_data={}
cal_data['face']=face_matrix
cal_data['mean_face']=np.mean(face_matrix,0)

cal_data['right_eye']=right_eye
cal_data['mean_right_eye']=np.mean(right_eye,0)

cal_data['left_eye']=left_eye
cal_data['mean_left_eye']=np.mean(left_eye,0)
cal_data['y']=y_cordinate

fp=open(pickle_dataname,'wb')
pickle.dump(file=fp,obj=cal_data)
fp.close()

In [ ]:
index=np.arange(face_matrix.shape[0])
np.random.shuffle(index)
tr_index=index[:int(index.shape[0]*0.7)]
te_index=index[int(index.shape[0]*0.7):]
val_left_eye,val_right_eye,val_face_matrix,val_y_cordinate=left_eye[te_index],right_eye[te_index],face_matrix[te_index],y_cordinate[te_index]
tr_left_eye,tr_right_eye,tr_face_matrix,tr_y_cordinate=left_eye[tr_index],right_eye[tr_index],face_matrix[tr_index],y_cordinate[tr_index]
y_cordinate=[]
left_eye=[]
right_eye=[]
face_matrix=[]

In [ ]:
tr_left_eye.shape,val_left_eye.shape

In [ ]:
mean_left_eye=np.mean(tr_left_eye,0)
mean_right_eye=np.mean(tr_right_eye,0)
mean_face_matrix=np.mean(tr_face_matrix,0)